# instal and load required libraries

The first step before starting to analyze the bulk RNA data is to install, and then load all the packges needed. To help us with that we will use BiocManager.  

In [ ]:
# install required packages
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install("GEOquery") 
BiocManager::install("Biobase") 
BiocManager::install("affy")
BiocManager::install("oligo")
BiocManager::install("biomaRt")
# install not-BiocManager packages
install.packages('umap')
install.packages('tidyverse')

In [ ]:
# load libraries
library(GEOquery)
library(Biobase)
library(affy)
library(oligo)
library(biomaRt)
library(tidyverse)

## Import dataset

The GEO dataset that we are going to use is GSE3790, from [Hodges et al 2006]([link](https://pubmed.ncbi.nlm.nih.gov/16467349/). In this study the authors used two platforms "GPL96" and "GPL97". For sake of time we will just consider "GPL96", and start from the code generated from NCBI geo2R. 
The Bioconductor packged for this platform is "Affyhgu133aExpr", and goes with the label "affy_hg_u133a_2" in biomaRt.
For more detail, here is the GEO [link](https://www.ncbi.nlm.nih.gov/geo/geo2r/?acc=GSE3790&platform=GPL96) for the data set's platform. 
Now we can get the data from this GEO obeject and began our analysis, which will be save in the `eset` object.

In [ ]:
# to ran annotation locally
## BiocManager::install("hgu133a2.db")
## BiocManager::install("Affyhgu133aExpr")
## library(Affyhgu133aExpr)
## library(hgu133a.db)
## library(hgu133a2.db)

# Download and get the expression set
query_GEO <- "GSE3790"
gset <- getGEO(query_GEO, GSEMatrix =TRUE, AnnotGPL = TRUE) #, getGPL=TRUE)
if (length(gset) > 1) idx <- grep("GPL96", attr(gset, "names")) else idx <- 1
eset <- gset[[idx]]

# to ran annotation locally
# add affymetrix chip platform annotation to eset
# eset@annotation <- 'Affyhgu133aExpr'

Now before we continue, lets us define some variable to be use througout the analysis. One of this is the `sampling_check`, which is use to quicly define subsetting ranges to check tables as we go. The second is the `platform_array`, just to avoid typos.

In [ ]:
# create parameter variables
## variable to subset array
sampling_check = c(1:5)
## array platform label
platform_array = 'affy_hg_u133a_2'

Now we can continue by matching the `probeNames` to the name of the genes. For that we used `biomaRt` package. 

In [ ]:
# Check probeNames
featureNames(eset)[sampling_check] # works
# oligo::probeNames(eset) # does not work

## get gene annotation from biomaRt
mart <- useMart("ENSEMBL_MART_ENSEMBL")
mart2 <- useDataset("hsapiens_gene_ensembl", mart)

# Create annotation loopk up object
## this is to match the probes name to their respective genes
annotLookup <- biomaRt::getBM(
  mart = mart2,
  attributes = c(
    platform_array,
    "ensembl_gene_id",
    "hgnc_symbol"
  ),
  filter= platform_array,
  values = rownames(exprs(eset)))

# exprs(set) average
xprs <-  exprs(eset)
## add column with probe names
xprs <- cbind(row.names(xprs),xprs)
## rename the column with affy chip's name
colnames(xprs)[1] <- platform_array
dim(xprs)
## add a column with the name of the gene from the biomaRt table
## remove the columns with 'str' data, only keep the numerical
xprs_annot <- dplyr::left_join(as.data.frame(xprs),annotLookup, by= platform_array)[,1-2]
dim(xprs_annot)

## Process data set

### Create a dataframe with summarized probes

Now we have too many probes for the less number of genes. To solve this, we looked to the boxplots in the GEO website, and we check that the data was already normalized. Therefore, for sake of time, we will just consider doing the `mean()` for all probes of each gene. Thus, we will loop through each gene (rows; features) and through each sample (cols; sample) and calculate the average of the values. The resulting dataframe will be the variable `xprs_processed`.
As control for this steps we will look at the dimension of the matrices, and we expect to end up with a less rows in the last data frame compared to the starting data frame---this is due to each gene corresponding to n-probes.
We should point out, that we are not interest in knowing the relative expression (healthy vs disease), because our main problem to solve is sample specific.

In [ ]:
# get the unique genes
repeated <- unique(xprs_annot[duplicated(xprs_annot$ensembl_gene_id),]$ensembl_gene_id)

# Initialize xprs_avg as a list ####
xprs_avg <- list()

# Make the average of different probes grouped by gene
## remove cols 202 and 203 that are strings
for (gene in repeated) {
  temp <- sapply(xprs_annot[xprs_annot$ensembl_gene_id == gene,-c(202,203)],  
                 function(x) {
                   mean(as.numeric(x), na.rm = TRUE)
                 })
  # Add temp to the list
  xprs_avg[[gene]] <- temp
}

# Convert the list to a data frame
xprs_avg <- do.call(rbind, xprs_avg)

# Convert matrix to data frame
xprs_avg <- as.data.frame(xprs_avg)

# bring back removed columns with string (genes)
## Add row names as a new column
xprs_avg$ensembl_gene_id <- rownames(xprs_avg)

# remove the duplicated from the original dataframe
xprs_unique <- xprs_annot[!xprs_annot$ensembl_gene_id %in% repeated,]
dim(xprs_unique)

# check if they are split => expected 0 True values
unique(xprs_unique$ensembl_gene_id) == repeated

# put ENSEMBL as the first column
xprs_avg <- xprs_avg[, c("ensembl_gene_id", setdiff(names(xprs_avg), "ensembl_gene_id"))]
xprs_unique <- xprs_unique[, c("ensembl_gene_id", setdiff(names(xprs_unique), "ensembl_gene_id"))][,-c(203)]
# create a new dataframe with averaged values with individual
xprs_processed <- rbind(xprs_avg,xprs_unique)

Now just for quality control, lets check the size of the resulting dataframes. If TRUE all is good. 

In [ ]:
# validate summarization
# the number of row names should be smaller
(length(repeated)+dim(xprs_unique)[1]) < dim(xprs) 
## the number of rows should be equal to the amount of unique genes
sum(length(repeated),dim(xprs_unique)[1]) == (length(unique(xprs_annot$ensembl_gene_id)))
## check final table
dim(xprs_processed)[1] == (length(unique(xprs_annot$ensembl_gene_id)))

## Export dataset

Now, the last touches. We need split the string in the annotation eset that contain the `age` and `condition` to match the matrix of training data from scRNA-seq. When this is done we can export as `.csv` to import into the other file where we are building the model.

In [ ]:
# get annotation data from eset
anno <- pData(eset)
head(anno)

Now we need to check the name of the columns that contains the string to be parsed, and add it to the function `select()`. 

In [ ]:
# subset the anno table for the specific columns that needs to be parsed 
anno2 <- select(anno,characteristics_ch1)
# Create new columns
anno2_split <- anno2 %>%
  mutate(condition = ifelse(str_detect(characteristics_ch1, "control"), "control", "HD"),
         condition_specified = str_extract(characteristics_ch1, "(control|grade \\d)"),
         age = as.numeric(str_extract(characteristics_ch1, "(?<=Age = )\\d+")),
         sex = str_extract(characteristics_ch1, "(?<=sex = )[MF]"))
anno2_split$source_name_ch1 <- anno$source_name_ch1 
anno2_split <-  select(anno2_split, -characteristics_ch1)
# Print the new data frame
print(anno2_split[sampling_check,])
row.names(anno2_split)[1]
row.names(anno2_split)[dim(anno2_split)[2]]
dim(anno2_split)
t_anno2_split <- as.data.frame(t(anno2_split))

Now that we have a dataframe with columns for each feature (age, condition ...) we need to merge it with the dataframe that has the mean values for each gene.

In [ ]:
# add a new "feature" to match the same n-columns as in xprs_processed
t_anno2_split$ensembl_gene_id <- row.names(t_anno2_split)
t_anno2_split <- t_anno2_split[, c("ensembl_gene_id", setdiff(names(t_anno2_split), "ensembl_gene_id"))]
dim(t_anno2_split)
# check if the colnames match
colnames(xprs_processed) == colnames(t_anno2_split)
# bind the rows of two data.frames
tissue_bulk_data <- rbind(xprs_processed,t_anno2_split)
dim(tissue_bulk_data)[1] == sum(dim(xprs_processed)[1] ,dim(t_anno2_split)[1] )

Finally, we have a dataframe ready to be exported and used to test the model with real data to which we want to know the % each cell type in the tissue.
See below parts of the final table.

In [ ]:
# print head
print(tissue_bulk_data[1:5,c(1,2,ncol(tissue_bulk_data))], row.names = FALSE)
# print tail
print(tissue_bulk_data[c((nrow(tissue_bulk_data)-5):nrow(tissue_bulk_data)),c(1,2,ncol(tissue_bulk_data))], row.names = FALSE)

In [ ]:
# export to csv
write.csv(tissue_bulk_data, file= "bulk_tissue_data.csv", row.names = FALSE)
save.image(file = "bulk_tissue_processed.Rdata")

# session info

In [ ]:
sessionInfo()